In [1]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import math

C:\Users\prroy\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [16]:
height = 32
width = 32
channels = 3

n_inputs = height * width * channels

conv1_filters = 96
conv1_ksize = 3
conv1_stride = 1
conv1_padding = 'SAME'

conv2_filters = 96
conv2_ksize = 3
conv2_stride = 2
conv2_padding = 'SAME'

conv3_filters = 192
conv3_ksize = 3
conv3_stride = 1
conv3_padding = 'SAME'

conv4_filters = 192
conv4_ksize = 3
conv4_stride = 2
conv4_padding = 'SAME'

conv5_filters = 192
conv5_ksize = 1
conv5_stride = 1
conv5_padding = 'VALID'

conv6_filters = 10
conv6_ksize = 1
conv6_stride = 1
conv6_padding = 'VALID'

n_outputs = 10

tf.reset_default_graph()

In [17]:
X = tf.placeholder(name='X', dtype=tf.float32, shape=(None, height, width, channels))
y = tf.placeholder(name='y', dtype=tf.int32, shape=(None))

conv1 = tf.layers.conv2d(X, filters=conv1_filters, kernel_size=conv1_ksize, strides=conv1_stride, padding=conv1_padding,
                        name='conv1', activation=tf.nn.relu)
conv2 = tf.layers.conv2d(conv1, filters=conv1_filters, kernel_size=conv1_ksize, strides=conv1_stride, padding=conv1_padding,
                        name='conv2', activation=tf.nn.relu)
conv3 = tf.layers.conv2d(conv2, filters=conv2_filters, kernel_size=conv2_ksize, strides=conv2_stride, padding=conv2_padding,
                        name='conv3', activation=tf.nn.relu)
drop1 = tf.layers.dropout(conv3, rate=0.5)
conv4 = tf.layers.conv2d(drop1, filters=conv3_filters, kernel_size=conv3_ksize, strides=conv3_stride, padding=conv3_padding,
                       name='conv4', activation=tf.nn.relu)
conv5 = tf.layers.conv2d(conv4, filters=conv3_filters, kernel_size=conv3_ksize, strides=conv3_stride, padding=conv3_padding,
                        name='conv5', activation=tf.nn.relu)
conv6 = tf.layers.conv2d(conv5, filters=conv4_filters, kernel_size=conv4_ksize, strides=conv4_stride, padding=conv4_padding,
                       name='conv6', activation=tf.nn.relu)
drop2 = tf.layers.dropout(conv6, rate=0.5)
conv7 = tf.layers.conv2d(drop2, filters=conv3_filters, kernel_size=conv3_ksize, strides=conv3_stride, padding=conv3_padding,
                        name='conv7', activation=tf.nn.relu)
conv8 = tf.layers.conv2d(conv7, filters=conv5_filters, kernel_size=conv5_ksize, strides=conv5_stride, padding=conv5_padding,
                        name='conv8', activation=tf.nn.relu)
conv9 = tf.layers.conv2d(conv8, filters=conv6_filters, kernel_size=conv6_ksize, strides=conv6_stride, padding=conv6_padding,
                        name='conv9')
pool1 = tf.layers.average_pooling2d(conv9, pool_size=8, strides=8, padding='VALID')

logits = tf.reshape(pool1, shape=(-1, 10))
y_prob = tf.nn.softmax(logits, name='y_prob')

learning_rate = 1e-4
#momentum = 0.9


with tf.name_scope('Train'):
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(cross_entropy)
    #optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=momentum )
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    training_op = optimizer.minimize(loss)

with tf.name_scope('eval'):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
init = tf.global_variables_initializer()

In [4]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [5]:
batches = []
batches.append(unpickle('../../../../prroy/datasets/CIFAR10/cifar-10-batches-py/data_batch_1'))
batches.append(unpickle('../../../../prroy/datasets/CIFAR10/cifar-10-batches-py/data_batch_2'))
batches.append(unpickle('../../../../prroy/datasets/CIFAR10/cifar-10-batches-py/data_batch_3'))
batches.append(unpickle('../../../../prroy/datasets/CIFAR10/cifar-10-batches-py/data_batch_4'))
batches.append(unpickle('../../../../prroy/datasets/CIFAR10/cifar-10-batches-py/data_batch_5'))
batches.append(unpickle('../../../../prroy/datasets/CIFAR10/cifar-10-batches-py/test_batch'))

In [6]:
def fetch_batch(batch):
    return batches[batch][b'data'].reshape((-1,3,32,32)).transpose((0,2,3,1)), np.array(batches[batch][b'labels'])

In [7]:
test_batch_data, test_batch_label = fetch_batch(5)

In [8]:
X_batch1, y_batch1 = fetch_batch(1)
X_batch2, y_batch2 = fetch_batch(2)
X_batch3, y_batch3 = fetch_batch(3)
X_batch4, y_batch4 = fetch_batch(4)
X_batch5, y_batch5 = fetch_batch(5)

In [9]:
X_data = np.concatenate((X_batch1, X_batch2, X_batch3, X_batch4, X_batch5))
X_data.shape

(50000, 32, 32, 3)

In [10]:
y_labels = np.concatenate((y_batch1, y_batch2, y_batch3, y_batch4, y_batch5))
y_labels.shape

(50000,)

In [12]:
X_train, X_val, y_train, y_val = train_test_split(X_data, y_labels, test_size=0.2, random_state=0)
print(X_train.shape, y_train.shape, X_val.shape, y_val.shape)

(40000, 32, 32, 3) (40000,) (10000, 32, 32, 3) (10000,)


In [13]:
def get_next_batch(starting_index, batch_size):
    return X_train[starting_index: min(starting_index + batch_size, len(X_train))],\
        y_train[starting_index: min(starting_index + batch_size, len(y_train))]

In [18]:
n_epochs = 100
batch_size = 128

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(math.ceil(len(X_train) / batch_size)):
            X_batch, y_batch = get_next_batch(iteration * batch_size, batch_size)
            sess.run(training_op, feed_dict = {X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_val = accuracy.eval(feed_dict={X: X_val, y: y_val})
        print('epoch: {}, training accuracy: {}, validation accuracy: {}'.format(epoch, acc_train, acc_val))
    print('Test set accuracy : {}'.format(accuracy.eval(feed_dict={X: test_batch_data, y: test_batch_label})))

epoch: 0, training accuracy: 0.421875, validation accuracy: 0.45399999618530273
epoch: 1, training accuracy: 0.484375, validation accuracy: 0.5425000190734863
epoch: 2, training accuracy: 0.546875, validation accuracy: 0.5945000052452087
epoch: 3, training accuracy: 0.625, validation accuracy: 0.6274999976158142
epoch: 4, training accuracy: 0.75, validation accuracy: 0.6541000008583069
epoch: 5, training accuracy: 0.765625, validation accuracy: 0.6661999821662903
epoch: 6, training accuracy: 0.765625, validation accuracy: 0.6820999979972839
epoch: 7, training accuracy: 0.796875, validation accuracy: 0.6955000162124634
epoch: 8, training accuracy: 0.828125, validation accuracy: 0.704200029373169
epoch: 9, training accuracy: 0.84375, validation accuracy: 0.7103000283241272
epoch: 10, training accuracy: 0.859375, validation accuracy: 0.7156000137329102
epoch: 11, training accuracy: 0.890625, validation accuracy: 0.7164000272750854
epoch: 12, training accuracy: 0.890625, validation accurac